In [ ]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Input, Dense
from keras.applications import ResNet50
from keras.applications.resnet import preprocess_input
print(keras.__version__)

2.11.0


In [ ]:
pip install split-folders[full]


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import splitfolders
input_folder = '/content/drive/MyDrive/rabies-data'
#######ration of split is 70%, 20% and 10%
splitfolders.ratio(input_folder, output="/content/drive/MyDrive/dataset", 
                   seed=42, ratio=(.7, .2, .1), 
                   group_prefix=None)


Copying files: 0 files [00:00, ? files/s]
Copying files: 1 files [00:00,  9.92 files/s]
Copying files: 13 files [00:00, 73.17 files/s]
Copying files: 24 files [00:00, 89.45 files/s]
Copying files: 33 files [00:00, 84.72 files/s]
Copying files: 45 files [00:00, 92.10 files/s]
Copying files: 60 files [00:00, 104.74 files/s]
Copying files: 74 files [00:00, 114.81 files/s]
Copying files: 86 files [00:00, 114.46 files/s]
Copying files: 98 files [00:00, 115.32 files/s]
Copying files: 110 files [00:01, 108.89 files/s]
Copying files: 121 files [00:01, 101.59 files/s]
Copying files: 134 files [00:01, 109.32 files/s]
Copying files: 146 files [00:01, 111.16 files/s]
Copying files: 159 files [00:01, 114.56 files/s]
Copying files: 171 files [00:01, 115.38 files/s]
Copying files: 183 files [00:01, 114.21 files/s]
Copying files: 197 files [00:01, 118.34 files/s]
Copying files: 212 files [00:01, 119.82 files/s]
Copying files: 227 files [00:02, 121.18 files/s]
Copying files: 242 files [00:02, 121.34 f

In [ ]:
num_classes = 2

image_resize = 224

batch_size_training = 100
batch_size_validation = 100
train_path = '/content/drive/MyDrive/dataset/train'
valid_path = '/content/drive/MyDrive/dataset/val'

In [ ]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [ ]:
train_generator = data_generator.flow_from_directory(
    train_path,
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 3278 images belonging to 2 classes.


In [ ]:
validation_generator = data_generator.flow_from_directory(
    valid_path,
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 936 images belonging to 2 classes.


In [ ]:
model = Sequential()

In [ ]:
model.add(ResNet50(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))

In [ ]:
# model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.layers

In [ ]:
model.layers[0].trainable = False

In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense_3 (Dense)             (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
steps_per_epoch_training = len(train_generator)/ batch_size_training
steps_per_epoch_validation = len(validation_generator)/ batch_size_validation
num_epochs = 2

In [ ]:
fit_history = model.fit(
    train_generator,
    # steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    # validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
33/33 [==============================] - 38s 1s/step - loss: 0.2491 - accuracy: 0.9097 - val_loss: 0.0617 - val_accuracy: 0.9797
Epoch 2/2
33/33 [==============================] - 32s 977ms/step - loss: 0.0504 - accuracy: 0.9896 - val_loss: 0.0321 - val_accuracy: 0.9936


In [ ]:
model.save('/content/drive/MyDrive/rabies_model.h5')

In [ ]:
from keras.models import load_model
pretrained_model_1 = load_model('/content/drive/MyDrive/rabies_model.h5')

In [ ]:
test_generator = data_generator.flow_from_directory(
    '/content/drive/MyDrive/dataset/test',
    target_size=(image_resize, image_resize),
    shuffle=False)

Found 471 images belonging to 2 classes.


In [ ]:
steps_per_epoch_test_2 = len(test_generator)/ batch_size_validation

In [ ]:
predict1 = pretrained_model_1.predict(test_generator)

15/15 [==============================] - 6s 334ms/step


In [ ]:
score_1 = pretrained_model_1.evaluate(test_generator)

15/15 [==============================] - 5s 235ms/step - loss: 0.0349 - accuracy: 0.9958


In [ ]:
# get the dictionary of classes
label2index = test_generator.class_indices

# obtain the list of classes
list_of_label = list(label2index.keys())
print("The list of classes: ", list_of_label)
print("ResNet50 Predictions :")
for i in range(0,471):
    pred_class = np.argmax(predict1[i])
    pred_label = list_of_label[pred_class]
    print('Prediction :{}'.format(
        pred_label,
        predict1[i][pred_class]))

The list of classes:  ['normal_dogs', 'rabid_dogs']
ResNet50 Predictions :
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :normal_dogs
Prediction :n